# Data Handling Workflows

### Importing Libraries and Configuration

We'll be using the htex configuration for Parsl. Read more [here.]( https://github.com/Parsl/parsl/blob/master/parsl/configs/htex_local.py)

In [1]:
import numpy as np

import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config

parsl.load(config)

### Map Reduce

A map reduce is a technique to execute multiple parallel jobs on a dataset to reduce the size of the dataset before executing a final function to get the result. A Map reduce is a more complicated version of synchronisation.

Let's consider a simple example where we are given multiple lists and we want to select the lists with the highest standard deviation.

![](./images/map_reduce.png)

In [2]:
# A python app to compute the standard deviation of the inputs
@python_app
def standard_deviation(inputs=[]):
    import numpy as np
    return np.std(inputs)

In [3]:
import random
# A function to construct data that is a list of lists, each each list having 100 random numbers

def make_data():
    lists = []
    
    for _ in range(100):
        new_list = []
        
        for __ in range(100):
            new_list.append(random.random()*100)
            
        lists.append(new_list)
    return lists

our_data = make_data()

In [4]:
# Computing the standard deviations for each list

standard_deviations = []

for i in our_data:
    standard_deviations.append(standard_deviation(i))

In [5]:
# Finding the maximum standard deviation

standard_deviations = [i.result() for i in standard_deviations]
print('Maximum Standard Deviation: ', max(standard_deviations))

Maximum Standard Deviation:  31.95613285570822


In [6]:
# Finding the list with the maximum standard deviation

maximum = max(standard_deviations)
print('Target List Number:', standard_deviations.index(maximum))

Target List Number: 40


# Hashing

We'll be using a simple hash function to store elements in our database. We'll evaluate the hash values in parallel and then store the items in those locations.

In [2]:
database = [0 for i in range(1000)]  # An empty database

In [3]:
@python_app 
def hash_function(element):
    import hashlib
    number = int(hashlib.md5(element).hexdigest()[:8], 16)%1000   # Creating a hash index
    return number

In [4]:
import random

elements = []

for i in range(100):
    element = '' 
    for _ in range(5):
        element += random.choice('abcdefghijklmopqrstuvwxyz')
    element = element.encode() # Making a 5 letter element
    elements.append(element) # collecting 100 such elements

In [5]:
hashes = []
for i in elements: # Updating the database for all the elements
    hashes.append(hash_function(i))

In [7]:
hashes = [i.result() for i in hashes]

In [8]:
for i in range(len(elements)):
    database[hashes[i]] = elements[i]

In [10]:
print(database)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'dsray', 0, b'xcbru', 0, b'pehqb', 0, 0, 0, 0, 0, 0, 0, 0, 0, b'cvxyx', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'ocvqj', 0, 0, 0, 0, 0, 0, 0, 0, b'xffqe', b'qfbvz', 0, 0, 0, 0, b'obujt', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'cjlzq', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'zcdah', 0, 0, 0, 0, 0, 0, 0, b'jybmt', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'egazm', 0, 0, b'irpma', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'eezjl', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'skrvr', 0, 0, 0, 0, 0, 0, b'qbfew', 0, 0, 0, 0, 0, 0, 0, b'kjate', 0, 0, 0, 0, 0, 0, 0, 0, 0, b'ejxzg', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'qrblt', 0, 0, 0, 0, b'fzfop', 0, 0, 0, 0, 0, 0, 0, 0, b'ixgim', 0, 0, 0, 0, b'fbpqk', 0, 0, 0, 0, 0, 0, b'lhczd', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b'yzemc', 0, 0, 0, 0, 0, 0, 0, 0, b'yclj

Note that this still doesn't solve the problem of overlap of elements. Chaining is the alternative here but in order to implement chaining, we have to evaluate the results which breaks the parallel thread.

## Montage Mosaic

### First Part

In [12]:
import montage_wrapper as montage

In [13]:
@bash_app
def unzip_images(stdout='echo-hello.stdout', stderr='echo-hello.stderr'):
    return 'tar xvf Kimages.tar'

In [15]:
unzip_images().result()

0

In [16]:
montage.mImgtbl('Kimages','Kimages.tbl')

stat : OK
count : 91
badfits : 0
badwcs : 0

In [20]:
montage.mMakeHdr('Kimages.tbl','Ktemplate.hdr')

stat : OK
msg : Cube columns exist but are either blank or inconsistent. Outputting 2D only.
count : 91
ncube : 0
naxis1 : 6110
naxis2 : 6857
clon : 275.185261
clat : -16.248875
lonsize : 1.697222
latsize : 1.904722
posang : 359.953803
lon1 : 276.072612
lat1 : -17.199892
lon2 : 274.296309
lat2 : -17.198517
lon3 : 274.306471
lat3 : -15.294203
lon4 : 276.065648
lat4 : -15.295564

In [44]:
os.mkdir('Kprojdir')

In [45]:
montage.mProjExec('Kimages.tbl','Ktemplate.hdr','Kprojdir/','stats.tbl')

stat : OK
count : 91
failed : 91
nooverlap : 0

In [32]:
montage.mImgtbl('Kprojdir/','images.tbl')

stat : OK
count : 0
badfits : 0
badwcs : 0

In [46]:
@bash_app
def mImgtbl(stdout='echo-hello.stdout', stderr='echo-hello.stderr'):
    return 'mImgtbl Kprojdir/ images.tbl'

In [47]:
mImgtbl().result()

0

In [ ]:
montage.mAdd( 'images.tbl' , 'Ktemplate.hdr', 'm17_uncorrected.fits')

In [51]:
@bash_app
def mViewer(stdout='echo-hello.stdout', stderr='echo-hello.stderr'):
    return 'mViewer -ct 1 -gray m17_uncorrected.fits -1s max gaussian-log -out m17_uncorrected.png'

### Second Part

In [ ]:
montage.mOverlaps('images.tbl','diffs.tbl')

In [ ]:
montage.mDiffExec('diffs.tbl', 'Ktemplate.hdr', 'diffdir')

In [ ]:
montage.mFitExec('diffs.tbl', 'fits.tbl', 'diffdir')

In [ ]:
montage.mBgModel('images.tbl', 'fits.tbl', 'corrections.tbl')

In [ ]:
montage.mBgExec('images.tbl', 'corrections.tbl', 'corrdir')

In [ ]:
montage.mAdd('images.tbl', 'Ktemplate.hdr', 'm17.fits')

In [ ]:
@bash_app
def mViewer2(stdout='echo-hello.stdout', stderr='echo-hello.stderr'):
    return 'mViewer -ct 1 -gray m17.fits -1s max gaussian-log -out m17.png'